Import the data

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls "/content/gdrive/My Drive/inaturalist_12K"

# set the path to the training and validation folders
train_path = '/content/gdrive/My Drive/inaturalist_12K/train'
val_path = '/content/gdrive/My Drive/inaturalist_12K/val'

Mounted at /content/gdrive
train  val


Install the dependencies

In [ ]:
!pip install wandb 

Import the libraries

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math
import matplotlib.pyplot as plt

In [4]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable = True) 

Setup to wandb

In [6]:
import wandb
from wandb.keras import WandbCallback 

In [7]:
wandb.login() 

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

 Preprocess the data

In [8]:
def dataProcess(batch_size = 32, data_aug = True, image_size = [299,299]):

    # data augmentation
    if(data_aug):
        train_datagen = ImageDataGenerator(
            rescale = 1./255,
            rotation_range = 90,
            shear_range = 0.2,
            zoom_range = 0.2,
            validation_split = 0.1,
            horizontal_flip = True)
    else:
        train_datagen = ImageDataGenerator( rescale = 1./255, validation_split = 0.1)

    test_datagen = ImageDataGenerator(rescale = 1./255)

    # generate train dataset
    train_generator = train_datagen.flow_from_directory(
        train_path,
        subset='training',
        target_size = image_size,
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True,
        seed = 45)

    # generate validation dataset
    val_generator = train_datagen.flow_from_directory(
        train_path,
        subset = 'validation',
        target_size = image_size,
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True,
        seed = 45)

    # generate test dataset
    test_generator = test_datagen.flow_from_directory(
        val_path,
        target_size = image_size,
        batch_size = batch_size,
        class_mode = 'categorical')

    '''
    # plot one image of each class
    class_names = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi',
               'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
    plt.figure(figsize=(10, 10))
    images, labels = val_generator.next()
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(class_names[np.where(labels[i] == 1)[0][0]])
        plt.axis("off")
    plt.show()
    '''
    return train_generator, val_generator, test_generator

Transfer learning and fine-tuning

In [9]:
def buildModel(pre_trained_model = 'InceptionV3',optimizer = 'adam', lr = 0.0001, image_size = [299,299], freeze = 1):
    print("image_size = ",image_size)
    
    # prepare the pre-trained model excluding the last layer
    if pre_trained_model == 'InceptionV3':
        pre_model = keras.applications.InceptionV3(input_shape = image_size + [3], weights = 'imagenet', include_top = False)
    if pre_trained_model == 'InceptionResNetV2':
        pre_model = keras.applications.InceptionResNetV2(input_shape = image_size + [3], weights = 'imagenet', include_top = False)
    if pre_trained_model == 'ResNet50':
        pre_model = keras.applications.ResNet50(input_shape = image_size + [3], weights = 'imagenet', include_top = False)
    if pre_trained_model == 'Xception':
        pre_model = keras.applications.Xception(input_shape = image_size + [3], weights = 'imagenet', include_top = False)
    
    '''
    # No fine tuning - keep the trained weights of the pre trained model same
    for layer in pre_model.layers:
        layer.trainable = False
    '''

    # fine tuning the model
    k = len(pre_model.layers)
    upto = math.ceil(k*freeze)
    for i in range(upto):
        pre_model.layers[i].trainable = False
    
    import os
    numberOfClasses = len(next(os.walk(train_path))[1])

    # flatten the last layer
    x = Flatten()(pre_model.output)
    # add output layer
    prediction = Dense(numberOfClasses, activation='softmax')(x)

    # create final model
    model = Model(inputs = pre_model.input, outputs = prediction)
    if optimizer == 'adam':
        opt = keras.optimizers.Adam(learning_rate = lr)
    if optimizer == 'adamax':
        opt = keras.optimizers.Adamax(learning_rate = lr)
    if optimizer == 'rmsprop':
        opt = keras.optimizers.RMSprop(learning_rate = lr)

    model.compile(optimizer = opt, 
                  loss = 'categorical_crossentropy', 
                  metrics = ['accuracy']
                  )
    return model

Sweep configurations

In [10]:
sweep_config = {
    'method' : 'random', 
    'metric' : {
        'name': 'val_accuracy',
        'goal': 'maximize'   
        },
    'parameters' : {
        'epochs': {'values' : [5,10]},
        'model' : {'values' : ['Xception', 'InceptionV3','InceptionResNetV2','ResNet50']},
        'optimizer' : {'values' : ['rmsprop','adam', 'adamax']},
        'batch_size' : {'values' : [16,32,64]},
        'lr' : {'values' : [1e-4,1e-5]},
        'data_aug' : {'values' : [True,False]},
        'freeze' : {'values' : [0.6,0.7,0.8,0.9]}
    }
}

In [11]:
def train():
    config_defaults = {
        "epochs" : 5,
        "model" : 'InceptionV3',
        "optimizer" : 'adamax',
        "batch_size" : 32,
        "lr" : 0.0001,
        "data_aug" : True,
        "freeze" : 1
    }

    wandb.init(config = config_defaults,name = "CS6910-A2-P_A")
    config = wandb.config
    wandb.run.name = "epochs_{}_model_{}_opt_{}_bs_{}_lr_{}_da_{}_freeze_{}".format(config.epochs,\
                                                                          config.model,\
                                                                          config.optimizer,\
                                                                          config.batch_size,\
                                                                          config.lr,\
                                                                          config.data_aug,\
                                                                          config.freeze
                                                                          )
    wandb.run.save()

    # resize the images as per the pre trained model
    image_size = [299,299]
    if config.model == 'ResNet50' :
        image_size = [224,224]

    # process the data
    train_set, val_set, test_set = dataProcess(config.batch_size, config.data_aug, image_size)

    # build model 
    model = buildModel(config.model, config.optimizer, config.lr, image_size, config.freeze)

    # train the model and save the history
    trained_model = model.fit(train_set,
                              steps_per_epoch = train_set.samples // config.batch_size,
                              validation_data = val_set, 
                              validation_steps = val_set.samples // config.batch_size,
                              epochs = config.epochs,
                              callbacks=[WandbCallback(monitor='val_accuracy',mode='max')]
                            )

    # evaluate the model
    model.evaluate(test_set,
                   batch_size = config.batch_size,
                   callbacks=[WandbCallback()]
                  )

    wandb.finish()
    return model, trained_model

In [ ]:
# generate sweep id
sweep_id = wandb.sweep(sweep_config,project = 'cs6910-As-2')

Create sweep with ID: piql1umf
Sweep URL: https://wandb.ai/pandp/cs6910-As-2/sweeps/piql1umf


In [ ]:
wandb.agent(sweep_id, train, count=40)
#wandb.agent('grwlicwk', function = train, project = 'cs6910-As-2', count=40)

wandb: Agent Starting Run: kb9fngde with config:
wandb: 	batch_size: 16
wandb: 	data_aug: True
wandb: 	epochs: 5
wandb: 	freeze: 0.9
wandb: 	lr: 0.0001
wandb: 	model: Xception
wandb: 	optimizer: adamax
wandb: Currently logged in as: pandp (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Found 9018 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
image_size =  [299, 299]
83697664/83683744 [==============================] - 2s 0us/step
Epoch 1/5
563/563 [==============================] - 4914s 9s/step - loss: 0.9286 - accuracy: 0.7018 - val_loss: 0.6502 - val_accuracy: 0.7883 - _timestamp: 1648967299.0000 - _runtime: 4948.0000
Epoch 2/5
563/563 [==============================] - 781s 1s/step - loss: 0.6135 - accuracy: 0.8018 - val_loss: 0.6057 - val_accuracy: 0.7974 - _timestamp: 1648968080.0000 - _runtime: 5729.0000
Epoch 3/5
563/563 [==============================] - 779s 1s/step - loss: 0.5183 - accuracy: 0.8294 - val_loss: 0.5605 - val_accuracy: 0.8155 - _timestamp: 1648968880.0000 - _runtime: 6529.0000
Epoch 4/5
563/563 [==============================] - 777s 1s/step - loss: 0.4319 - accuracy: 0.8573 - val_loss: 0.5539 - val_accuracy: 0.8155 - _timestamp: 1648969682.0000 - _runtime: 7331.0000
Epoc

accuracy,▁▅▆▇█
epoch,▁▃▅▆█
loss,█▄▃▂▁
val_accuracy,▁▃▆▆█
val_loss,█▅▃▂▁
accuracy,0.87347
best_epoch,4
best_val_accuracy,0.82762
epoch,4
loss,0.38097
val_accuracy,0.82762


wandb: Agent Starting Run: hqh5atdn with config:
wandb: 	batch_size: 16
wandb: 	data_aug: True
wandb: 	epochs: 5
wandb: 	freeze: 0.8
wandb: 	lr: 1e-05
wandb: 	model: Xception
wandb: 	optimizer: adam


Found 9018 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
image_size =  [299, 299]
Epoch 1/5
117/563 [=====>........................] - ETA: 9:42 - loss: 1.7509 - accuracy: 0.4252